In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [19]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.matmul(x, W) + b

# Normalize
sm = tf.nn.softmax(y)

# Cross entropy
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(sm), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

for _ in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
print "Trained!"

Trained!


In [20]:
correct_prediction = tf.equal(tf.argmax(sm, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9189


In [37]:
test = [ 784 * [0.5] ]
sess.run(sm, {x : test})

array([[  4.29075939e-04,   4.39868586e-08,   4.04969305e-01,
          3.63606542e-01,   3.27174035e-07,   2.18060330e-01,
          3.98885313e-04,   1.01242942e-04,   1.24046579e-02,
          2.95190930e-05]], dtype=float32)

In [56]:
from PIL import Image

inputfile = "1.jpg"
outputfile = "output.jpg"

img = Image.open(inputfile)
img = img.convert(mode="L")
img = img.resize((28, 28), Image.ANTIALIAS)
raster_data = list()
for row in range(28):
    for col in range(28):
        raster_data.append(img.getpixel((row, col)))

xs = [ raster_data ]
        
result = sess.run(sm, { x:  xs })
result = result[0]

print result

max = 0
for digit in range(10):
    if result[digit] > result[max]:
        max = digit
        
print max

img.save(outputfile)

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
3
